In [ ]:
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dropout, Dense
from keras.optimizers import Adam
from keras import backend as K
import tensorflow as tf
from keras.losses import categorical_crossentropy, kullback_leibler_divergence
from keras.callbacks import TensorBoard
%matplotlib inline
from matplotlib import pylab as pl

In [ ]:

X_train = np.load('sample_data.npy')
X_train.shape

In [ ]:
def generate_signal_naive(my_sequence, length=250):
    from scipy.stats import norm

    base_strength = [4, 3, 2, 1]
    idx = my_sequence.argmax(1).reshape(my_sequence.shape[0],-1)
    X = np.zeros((my_sequence.shape[0], 1, length, 1))
    
    for sample_no in range(my_sequence.shape[0]):
        for ix in range(my_sequence.shape[2]):
            base = idx[sample_no, ix]
            X[sample_no, 0, ix*10:(ix*10+5), 0] +=  base_strength[base]
    return X

def generate_signal(my_sequence, length=250, channels=3):
    from scipy.stats import norm

    base_strength = [[4, 3, 2, 1], [2, 3, 4, 1], [1, 4, 3, 2]]
    base_noise = [.02, .01, .02, .01]
    idx = my_sequence.argmax(1).reshape(my_sequence.shape[0],-1)
    X = np.zeros((my_sequence.shape[0], channels, length, 1))
    
    for sample_no in range(my_sequence.shape[0]):
        for ix in range(my_sequence.shape[2]):
            base = idx[sample_no, ix]
            for channel in range(channels):
                mu = norm.rvs(loc=10, scale=2)
                sigma = abs(norm.rvs(loc=10, scale=1))
                amplitude = norm.rvs(loc=base_strength[channel][base], scale=base_noise[base])
                X[sample_no, channel, :, 0] += amplitude*norm.pdf(np.arange(length), loc=mu+ix*10, scale= sigma)
    return X

def random_crop(template_dna, size=10):
    ix = np.random.randint(0,template_dna.shape[2]-size, template_dna.shape[0])
    cropped = np.zeros((template_dna.shape[0], template_dna.shape[1], size))
    for i in range(len(ix)):
        cropped[i] = template_dna[i, :, ix[i]:(ix[i]+size)].squeeze()
    return cropped

def get_batch(X_train, batch_size=50):
    ix = -batch_size
    while True:
        if ix>=(X_train.shape[0]-batch_size):
            ix = -batch_size
        ix+=batch_size
        template = X_train[ix:(ix+batch_size)]
       
        dna = random_crop(template, size=10)
        signal = generate_signal(dna, length=100)
        
        yield  template, dna, signal

        
# Spare some data for validation
template = X_train[:500]
val_dna = random_crop(template, size=10)
val_signal = generate_signal(val_dna, length=100)

In [ ]:
x = random_crop(X_train[:2,:,:,:], size=10)


In [ ]:
## Rerun this code to see how the same DNA seq can give very different profiles
x_ = generate_signal(x, length=100)
pl.plot(x_[0,0,:,0])
pl.plot(x_[0,1,:,0])
pl.plot(x_[0,2,:,0])
x[0]

### ConvLSTMs

In [ ]:

# Parameters
learning_rate = 0.001
training_iters = 100000
batch_size = 128
display_step = 10

# Network Parameters
n_input = 32 
#meta timesteps
n_steps = 10 # timesteps 
n_hidden = 50 # hidden layer num of features
n_classes = 4 # bases

#predict all bases from the signal or just the leftmost base
seq_pred = False

In [ ]:
tf.reset_default_graph()

## input signal with 3 features, 100 timepoints
inp = tf.placeholder(tf.float32, [None, 3, 100, 1])
if seq_pred:
    targets = tf.placeholder(tf.float32, [None, n_classes, n_steps])
    list_targets = tf.unstack(targets, n_steps, 2)
else:
    targets = tf.placeholder(tf.float32, [None, n_classes])
    
# Define weights
weights = {
    'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
}
biases = {
    'out': tf.Variable(tf.random_normal([n_classes]))
}



In [ ]:

def RNN(x, weights, biases, return_seq=False):

    # input shape: (batch_size, n_steps, n_input)
    # converted shape:'n_steps' tensors list of shape (batch_size, n_input)

    # Unstack to get a list of 'n_steps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, n_steps, 1)
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    outputs, states = tf.contrib.rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

    if return_seq:
        return [tf.matmul(outp, weights['out']) + biases['out'] for outp in outputs]
    else:
        return tf.matmul(outputs[-1], weights['out']) + biases['out']


In [ ]:
net = Conv2D(32, [3, 20],
                padding='same',
                name='conv_1')(inp)
net = tf.reduce_sum(net, axis=1, keep_dims=True)
net = MaxPooling2D((1, 10), strides=(1, 10))(net)
net = tf.squeeze(net, squeeze_dims=1)

preds = RNN(net, weights, biases, return_seq=seq_pred)


In [ ]:

if seq_pred:
    losses = [tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=target) 
              for pred,target in zip(preds, list_targets)]
    correct_pred = [tf.equal(tf.argmax(pred,1), tf.argmax(target,1)) 
                    for pred,target in zip(preds, list_targets)]
else:
    losses = tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=targets)
    correct_pred = tf.equal(tf.argmax(preds,1), tf.argmax(targets,1))
    
# Define loss and optimizer
cost = tf.reduce_mean(losses)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Evaluate model

accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))


In [ ]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [ ]:
# For tensorboard viz


if seq_pred:
    run_name = 'convLstm_seq'
else:
     run_name = 'convLstm'

tf.summary.scalar('classification_cost', cost)
tf.summary.scalar('Accuracy', accuracy)

summary_op = tf.summary.merge_all()
summary_writer_train = tf.summary.FileWriter( run_name+'/train', sess.graph)
summary_writer_valid = tf.summary.FileWriter( run_name+'/validation', sess.graph)



In [ ]:
# from terminal type:
# tensorboard --logdir=.

In [ ]:
batcher = get_batch(X_train[500:], batch_size=batch_size)
step=0
for i in range(training_iters):
    for j in range(10):
        _, dna, signal = batcher.next()
        if seq_pred:
            y_target = dna #predict all 10 bases
            y_valid = val_dna
        else:
            y_target = dna[:,:,0] # predict only the leftmost base
            y_valid = val_dna[:,:,0]
            
        _ = sess.run(optimizer, feed_dict={inp:signal, targets:y_target, K.learning_phase():1})
    
    np_acc_tr, np_loss_tr, train_summary = sess.run([accuracy, cost, summary_op], 
                                  feed_dict={inp:signal, targets: y_target, K.learning_phase():0})
    np_acc_vl, np_loss_vl, validation_summary = sess.run([accuracy, cost,  summary_op], 
                                  feed_dict={inp:val_signal, targets: y_valid, K.learning_phase():0})
    print('Iteration {}\t train_loss:{:.4f}\t val_loss:{:.4f}\t  \
    train_acc:{:.2f}\t val_acc:{:.2f}\t'.format(i, np_loss_tr, np_loss_vl, np_acc_tr, np_acc_vl))
    summary_writer_train.add_summary(train_summary, step)
    summary_writer_valid.add_summary(validation_summary, step)
    summary_writer_train.flush()
    summary_writer_valid.flush()
    step+=1